# Code Generator

## json을 통해서 verilog code를 생성하는 기능을 구현하고자 합니다. 

---

# protocol gen check

In [1]:
import json
from pprint import pprint

In [2]:
my_p={}
with open("./protocol/my_protocol.json") as f:
    my_p = json.load(f)

In [3]:
pprint(my_p)

{'master': [{'direction': 'input', 'name': 'CLK', 'width': 1},
            {'direction': 'input', 'name': 'RESETn', 'width': 1},
            {'direction': 'output', 'name': 'VALID', 'width': 1},
            {'direction': 'input', 'name': 'READY', 'width': 1},
            {'direction': 'output', 'name': 'O_DATA', 'width': 32},
            {'direction': 'input', 'name': 'I_DATA', 'width': 32}],
 'name': 'my_protocol',
 'slave': [{'direction': 'input', 'name': 'CLK', 'width': 1},
           {'direction': 'input', 'name': 'RESETn', 'width': 1},
           {'direction': 'input', 'name': 'VALID', 'width': 1},
           {'direction': 'output', 'name': 'READY', 'width': 1},
           {'direction': 'output', 'name': 'O_DATA', 'width': 32},
           {'direction': 'input', 'name': 'I_DATA', 'width': 32}]}


In [4]:
with open("./gen_code.v","w")as f:
    for i in my_p['slave']:
        line_code = "\t."+i['name']+"\t\t(\t)\n"
        f.writelines(line_code)   

# 코드창에 !를 입력하고 명령어를 넣으면 bash 명령어가 실행 됩니다.

### !cat ./gen_code.v 를 통해서 생성된 code를 확인해보겠습니다. 

In [5]:
!cat ./gen_code.v

	.CLK		(	)
	.RESETn		(	)
	.VALID		(	)
	.READY		(	)
	.O_DATA		(	)
	.I_DATA		(	)


## json load

json을 부를때는 json자체에 적힌 순서대로 읽어 오기위해서 

아래와 같이 **object_pairs_hook=OrderedDict** 옵션을 주어야 합니다.



```python
json.load(f,object_pairs_hook=OrderedDict)
```



In [6]:
from collections import OrderedDict
top_info = {}
with open("./info/top.json") as f:
    #top_info = json.load(f)
    top_info = json.load(f,object_pairs_hook=OrderedDict)

In [7]:
pprint(top_info)

OrderedDict([('top_name', 'top'),
             ('port_list',
              [OrderedDict([('port_name', 'CLK'),
                            ('direction', 'input'),
                            ('width', ''),
                            ('prefix', 'i_'),
                            ('postfix', '_top')]),
               OrderedDict([('port_name', 'RESETn'),
                            ('direction', 'input'),
                            ('width', ''),
                            ('prefix', 'i_'),
                            ('postfix', '_top')]),
               OrderedDict([('port_name', 'DATA'),
                            ('direction', 'input'),
                            ('width', '[31:0]'),
                            ('prefix', 'i_'),
                            ('postfix', '_top')]),
               OrderedDict([('port_name', 'DATA'),
                            ('direction', 'output'),
                            ('width', '[31:0]'),
                            ('prefix', 'o_'),
    

# port_list code generator

In [8]:
def top_module_open(f,top_info):
    f.writelines("module "+top_info['top_name']+" (\n" )

def top_module_close(f):
    f.writelines(");\n")

def endmodule(f):
    f.writelines("endmodule")
    
def make_port_string(f,info_dict):
    if(info_dict['width']==''):
        f.writelines("\t"+info_dict['direction'] +"\t\t\t\t" + info_dict['prefix']+info_dict['port_name']+info_dict['postfix']+",\n")
    else:
        f.writelines("\t"+info_dict['direction'] +"\t" + info_dict['width'] + "\t\t" + info_dict['prefix']+info_dict['port_name']+info_dict['postfix']+",\n")
    

In [9]:
with open("./gen_code.v","w")as f:
    top_module_open(f,top_info)
    for i in top_info['port_list']:
        make_port_string(f,i)
    top_module_close(f)
    endmodule(f)

---

In [10]:
!cat ./gen_code.v

module top (
	input				i_CLK_top,
	input				i_RESETn_top,
	input	[31:0]		i_DATA_top,
	output	[31:0]		o_DATA_top,
);
endmodule

---

# connection_list 만들기